## Build Enviroment

In [1]:
!pip install -r requirements.txt

ERROR: Could not find a version that satisfies the requirement torch==1.6.0 (from versions: 1.7.1, 1.8.0, 1.8.1, 1.9.0, 1.9.1, 1.10.0, 1.10.1, 1.10.2, 1.11.0, 1.12.0, 1.12.1)
ERROR: No matching distribution found for torch==1.6.0


# Train a model

To train a model you should specify the arguments, for example:

```bash
python run_metonymy_resolution.py \
--data_dir ../data \
--train_file conll_train.json \
--predict_file conll_test.json \
--output_dir ../output \
--do_train  \
--do_eval \
--do_mask
```

# You can also use our pretrained model

Download from, https://drive.google.com/file/d/1PCXkEFyK5OALQbF_64J6jSGO0IUjbYuf/view?usp=sharing

Unzip and put it to ``model_folder`` (you local path).

This is a pretrained bert-base-uncased model, use ./data/conll_train.json.

# Load pretrained model from ``model_folder``

In [1]:
import numpy as np
import sys
sys.path.append("./src")
from utils_metonymy import *

model_class = BertForWordClassification
tokenizer_class = BertTokenizer

In [2]:
model_folder = './output' # specify which dir the model been saved.

model = model_class.from_pretrained(model_folder)
tokenizer = tokenizer_class.from_pretrained(model_folder, do_lower_case=True)

# Input example

In [3]:
example = {'sentence': ['SOCCER', '-', 'ROMANIA', 'BEAT', 'LITHUANIA', 'IN', 'UNDER-21', 'MATCH.'],
           'pos': [2, 3]}

inputs = convert_single_example_to_input(example, tokenizer)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


# Run the model on the input example

In [4]:
model.zero_grad()
model.eval()

logits = model(**inputs)[0]


preds = logits.detach().cpu().numpy()
preds = np.argmax(preds, axis=1)

label_map = {0:'literal',1:'metonymy'}

print(f'Target word:   {" ".join(example["sentence"][example["pos"][0]:example["pos"][1]])}')
print(f'Sentence:      {" ".join(example["sentence"])}')
print(f'Prediction:    {label_map[preds[0]]}')

Target word:   x
Sentence:      SOCCER - x BEAT LITHUANIA IN UNDER-21 MATCH.
Prediction:    metonymy
